In [16]:
%pip install constants

Note: you may need to restart the kernel to use updated packages.


In [17]:
%pip install 'PyPDF2<3.0'

Note: you may need to restart the kernel to use updated packages.


In [18]:
%pip uninstall torch transformers -y

Note: you may need to restart the kernel to use updated packages.


In [19]:
%pip install transformers

  Using cached transformers-4.51.3-py3-none-any.whl.metadata (38 kB)
Using cached transformers-4.51.3-py3-none-any.whl (10.4 MB)
Note: you may need to restart the kernel to use updated packages.


In [20]:
%pip install torch

  Using cached torch-2.7.0-cp311-cp311-manylinux_2_28_x86_64.whl.metadata (29 kB)
Using cached torch-2.7.0-cp311-cp311-manylinux_2_28_x86_64.whl (865.2 MB)
Note: you may need to restart the kernel to use updated packages.


In [2]:
import pickle
import nltk
import re
import pandas as pd
import numpy as np
import constants as const
import PyPDF2
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.stem.porter import PorterStemmer
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from transformers import BertModel, BertTokenizer
import torch
nltk.download(['punkt_tab', 'stopwords', 'wordnet'])

/home/zeuidon/Documents/CourseWork/Mini-project/.venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
[nltk_data] Downloading package punkt_tab to
[nltk_data]     /home/zeuidon/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.
[nltk_data] Downloading package stopwords to
[nltk_data]     /home/zeuidon/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /home/zeuidon/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [3]:
def tfidf_similarity(tokenized_text1, tokenized_text2):
    tfidf_vectorizer = TfidfVectorizer()
    tfidf_matrix = tfidf_vectorizer.fit_transform([tokenized_text1, tokenized_text2])

    # Similarity Measurement
    cosine_sim = cosine_similarity(tfidf_matrix[0], tfidf_matrix[1])

    return cosine_sim[0][0]

In [4]:
print(torch.__version__)

2.7.0+cu126


In [5]:
model_name = "bert-base-uncased"
tokenizer = BertTokenizer.from_pretrained(model_name)
model = BertModel.from_pretrained(model_name)

In [6]:
def compute_bert_embeddings(sentence, model, tokenizer):
    input_ids = tokenizer.encode(sentence, add_special_tokens=True)
    input_ids = torch.tensor(input_ids).unsqueeze(0)
    with torch.no_grad():
        output = model(input_ids)
    return output.last_hidden_state.mean(dim=1).numpy()

In [7]:
def combine_embeddings(long_text):
  max_segment_length = 512  # Maximum segment length allowed by BERT
  segments = [long_text[i:i + max_segment_length] for i in range(0, len(long_text), max_segment_length)]

  # Initialize an empty tensor for aggregated embeddings
  aggregated_embeddings = torch.zeros((1, 768))  # Ensure the size matches BERT's embedding size (e.g., 768)

  # Process each segment separately and aggregate embeddings
  for segment in segments:
      encoded_input = tokenizer(segment, return_tensors='pt', padding=True, truncation=True, max_length=512)
      with torch.no_grad():
          output = model(**encoded_input)
      segment_embedding = output.last_hidden_state.mean(dim=1)  # Mean pooling over tokens
      aggregated_embeddings += segment_embedding
  return aggregated_embeddings

In [8]:
def bert_similarity(tokenized_text1, tokenized_text2) :
    embeddings1 = combine_embeddings(tokenized_text1)
    embeddings2 = combine_embeddings(tokenized_text2)

    # Calculate cosine similarity
    return cosine_similarity(embeddings1, embeddings2)[0][0]

In [8]:
%pip install fastapi

  Using cached annotated_types-0.7.0-py3-none-any.whl.metadata (15 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 2.0 MB/s eta 0:00:00 MB/s eta 0:00:01:01
Using cached annotated_types-0.7.0-py3-none-any.whl (13 kB)
Note: you may need to restart the kernel to use updated packages.


In [9]:
%pip install pydantic

Note: you may need to restart the kernel to use updated packages.


In [2]:
%pip install uvicorn

Note: you may need to restart the kernel to use updated packages.


In [9]:
from fastapi import FastAPI
from pydantic import BaseModel

class Item(BaseModel):
    t1: str
    t2: str

app = FastAPI()

@app.post("/in")
def create(item: Item):
    dt = item.dict()
    text1 = dt['t1']
    text2 = dt['t2']
    b = str(bert_similarity(text1, text2))
    t =  str(tfidf_similarity(text1, text2))
    return {
        "Bert similarity": b, 
        "Tfidf similarity": t
    }

import nest_asyncio
import uvicorn

if __name__ == "__main__":
    nest_asyncio.apply()
    uvicorn.run(app)

INFO:     Started server process [26018]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://127.0.0.1:8000 (Press CTRL+C to quit)
/tmp/ipykernel_26018/3578197039.py:12: PydanticDeprecatedSince20: The `dict` method is deprecated; use `model_dump` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.11/migration/
  dt = item.dict()


INFO:     127.0.0.1:33858 - "POST /in HTTP/1.1" 200 OK


INFO:     Shutting down
INFO:     Waiting for application shutdown.
INFO:     Application shutdown complete.
INFO:     Finished server process [26018]
